In [ ]:
from enum import Enum
import bs4 as bs

class VocabularyXMLFileType(Enum):
    XML_BIREME = 'Bireme'
    XML_ERIC_THESAURUS = 'Eric'
    SKOS = 'SKOS'

class VocabularyLabel:
    def __init__(self, label:str, preferential_label: str = None, uri: str = None):
        self.label = label
        self.preferential_label = preferential_label
        self.uri = uri

class Vocabulary:
    def loadFromXmlFile(self, file_path: str, lang : str = 'en', file_type : VocabularyXMLFileType = VocabularyXMLFileType.SKOS):
        xml_data = None
        with open(file_path, 'r') as xml_file:
            xml_data = xml_file.read()
        xml_soup = bs.BeautifulSoup(''.join(xml_data), "xml")
        
        content = {}

        for element in xml_soup.find_all('rdf:Description'):
            pref = element.find('skos:prefLabel', {'xml:lang' : lang})
            if pref:
                content[' '.join(pref.contents)] = None
                self._appendLabel(VocabularyLabel(' '.join(pref.contents), ' '.join(pref.contents)))
                for alt in element.find_all('skos:altLabel'):
                    if alt['xml:lang'] == lang:
                        content[' '.join(alt.contents)] = ' '.join(pref.contents)
                        self._appendLabel(VocabularyLabel(' '.join(alt.contents), ' '.join(pref.contents)))
        self.contents = content
    
    def generateCompatibleSisaFile(self, file_path:str):
        with open(file_path, 'w') as out_file:
            for k, v in dict(sorted(self.contents.items())).items():
                out_file.write(k + "\n")
                if v != None:
                    out_file.write("\tUSE " + v + "\n")
    
    def generateTSVFile(self, file_path: str):
        return ''
    
    def getPreferentialLabel(self, label: str, ignore_case: bool = True):
        label_obj = self.getLabel(label, ignore_case)
        result = None if label_obj == None else label_obj.preferential_label
        return result
    
    def getLabelUri(self, label: str, ignore_case: bool = True):
        label_obj = self.labels_lowercase[label.lower()] if ignore_case else self.labels[label]
        result = None if label_obj == None else result.uri
        return result
    
    def getLabel(self, label: str, ignore_case: bool = True):
        obj_label = None
        if ignore_case:
            obj_label = self.labels_lowercase[label.lower()] if label.lower() in self.labels_lowercase else None
        else:
            obj_label = self.labels[label] if label in self.labels else None
        return obj_label
    
    def _appendLabel(self, label: VocabularyLabel):
        self.labels[label.label] = label
        self.labels_lowercase[label.label.lower()] = label
    
    def __init__(self):
        self.name = ''
        self.contents = {}
        self.origin = None
        self.labels = {}
        self.labels_lowercase = {}